In [2]:
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands

In [19]:
def calculate_indicators(df: pd.DataFrame, price_col: str = "price") -> pd.DataFrame:
    indicators = pd.DataFrame()

    # log return
    indicators["log_ret"] = np.log(df[price_col] / df[price_col].shift(1))

    # RSI
    rsi = RSIIndicator(close=df[price_col], window=14)
    indicators["rsi"] = rsi.rsi()

    # MACD
    macd = MACD(close=df[price_col], window_slow=26, window_fast=12, window_sign=9)
    indicators["macd"] = macd.macd_diff()  # можно использовать .macd() и .macd_signal()

    # Moving Averages
    indicators["ma_fast"] = df[price_col].rolling(5).mean()
    indicators["ma_slow"] = df[price_col].rolling(20).mean()
    indicators["ma_diff"] = indicators["ma_fast"] - indicators["ma_slow"]

    # Bollinger Bandwidth
    bb = BollingerBands(close=df[price_col], window=20, window_dev=2)
    indicators["bb_width"] = bb.bollinger_hband() - bb.bollinger_lband()

    # Rolling STD (волатильность)
    indicators["volatility"] = df[price_col].rolling(window=10).std()

    return indicators

In [26]:
input_file = "../data/prices/BTC_500ms.csv"
output_file = "../data/indicators/BTC_500ms.csv"

df = pd.read_csv(input_file)
indicators = calculate_indicators(df)

In [27]:
indicators.to_csv(output_file, index = False)